# Advent of Code 2018

See [here](http://adventofcode.com/2018/).

## Preparation

Imports and utility functions that might or might not prove useful down the line.

In [42]:
# Python 3.x
import re
import numpy as np
import math
import random
import urllib.request
import reprlib
import operator
import string
import hashlib
import json

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache, reduce
from itertools   import (permutations, combinations, chain, cycle, tee,
                        product, islice, count, repeat, filterfalse, accumulate)
from heapq       import heappop, heappush
from enum        import Enum

# sometimes a graph helps
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (8,8)

def Input(day,strip=True):
    "Open this day's input file."
    
    filename = 'input/input{}.txt'.format(day)
    try:
        with open(filename, 'r') as f:
            text = f.read()
            if strip:
                text = text.strip()
        return text
    except FileNotFoundError:
        url = 'http://adventofcode.com/2018/day/{}/input'.format(day)
        print('input file not found. opening browser...')
        print('please save the file as "input<#day>.txt in your input folder.')
        import webbrowser
        webbrowser.open(url)

cat = ''.join
def first(iterable, default=None): return next(iter(iterable), default)
def nth(iterable, n, default=None): return next(islice(iterable, n, None), default)
def fs(*items): return frozenset(items)

def window(seq, n=2):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(seq)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result

def ilen(iterator): return sum(1 for _ in iterator)

def ints(text,typ=int):
    return list(map(typ,re.compile(r'[-+]?\d*[.]?\d+').findall(text)))

def shift(it, n):
    return it[n:] + it[:n]

def rot(mat, N=1, clockwise=True):
    '''rotate 2D matrix'''
    for _ in range(N):
        if clockwise:
            mat = list(zip(*mat[::-1]))
        else:
            mat = list(zip(*mat[::-1]))[::-1]
    return mat

def locate2D(m, val):
    '''locate value in 2D list'''
    for i, line in enumerate(m):
        j=-1
        try:
            j = line.index(val)
        except ValueError:
            continue
        break
    else:
        i = -1
    return (i,j)

def dist_L1(p1,p2=None):
    if p2 == None:
        p2 = repeat(0)
    return sum(abs(p2_i-p1_i) for p1_i, p2_i in zip(p1,p2))

def dist_L2(p1,p2=None):
    if p2 == None:
        p2 = repeat(0)
    return sum((p2_i-p1_i)*(p2_i-p1_i) for p1_i, p2_i in zip(p1,p2))**.5

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

from numbers import Number 
class Vector(object):
    def __init__(self,*args):
        if len(args) == 1:
            if isinstance(args,Number): self.vec = tuple(0 for _ in range(args))
            else: self.vec = tuple(*args)
        else: self.vec = tuple(args)
    def __mul__(self, other):
        if isinstance(other,Number): return Vector(other * x for x in self.vec)
        elif isinstance(other,Vector): return sum(x*y for x,y in zip(self.vec, other.vec))
        raise NotImplemented
    def __add__(self,other):
        return Vector(x+y for x,y in zip(self.vec, other.vec))
    def __sub__(self,other):
        return Vector(x-y for x,y in zip(self.vec, other.vec))
    def __iter__(self):
        return self.vec.__iter__()
    def __len__(self):
        return len(self.vec)
    def __getitem__(self, key):
        return self.vec[key]
    def __repr__(self):
        return 'Vector(' + str(self.vec)[1:-1] + ')'
    def __eq__(self, other):
        return self.vec == other.vec
    def __hash__(self):
        return hash(self.vec)

#display and debug functions
def h1(s):
    upr, brd, lwr = '▁', '█', '▔'
    return upr*(len(s)+4) + '\n'+brd+' ' + s + ' ' + brd +'\n' + lwr*(len(s)+4)

def h2(s, ch='-'):
    return s + '\n' + ch*len(s) + '\n'

h1 = lambda s: h2(s,'=')  #the other h1 is a bitch, apparently.

def print_result(day, part, text):
    print(h1('Day {} part {}: {}'.format(day, part, text)))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    rep = reprlib.aRepr
    rep.maxother = 85
    def traced_f(*args):
        arg_strs = ', '.join(map(rep.repr, args))
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, arg_strs, result))
        return result
    return traced_f

## [Day 1: Chronal Calibration](http://adventofcode.com/2018/day/1)

In [2]:
data = Input(1)
frequencies = ints(data)

day1_part1 = sum

def day1_part2(frequencies):
    r = 0
    s = set([r])
    for f in cycle(frequencies):
        r += f
        if r in s:
            return r
        s.add(r)

In [3]:
print_result(1,1,'Frequency is '+str(day1_part1(frequencies)))
print_result(1,2,'first double is ' + str(day1_part2(frequencies)))

Day 1 part 1: Frequency is 439

Day 1 part 2: first double is 124645



## [Day 2: Inventory Management System](http://adventofcode.com/2018/day/2)

In [4]:
data = Input(2).split()

def day2_part1(ids):
    """computing the checksum"""
    data = [Counter(s) for s in ids]
    data = [list(set(v for k,v in s.items() if v == 2 or v == 3)) for s in data if 2 in s.values() or 3 in s.values()]
    data = Counter(sum(data,[]))
    checksum = data[2]*data[3]
    return checksum

test_p1 = ['abcdef', 'bababc', 'abbcde', 'abcccd', 'aabcdd', 'abcdee', 'ababab']
assert day2_part1(test_p1) == 12

def day2_part2(ids):
    """find common letters between matching IDs"""
    for id_1, id_2 in combinations(ids,2):
        common = cat(c_1 for c_1, c_2 in zip(id_1, id_2) if c_1 == c_2)
        if len(common) + 1 == len(id_1):
            return common

test_p2 = ['abcde', 'fghij', 'klmno', 'pqrst', 'fguij', 'axcye', 'wvxyz'] 
assert day2_part2(test_p2) == 'fgij'

In [5]:
print_result(2,1,'checksum is '+str(day2_part1(data)))
print_result(2,2,'common letters for the matching IDs are "'+str(day2_part2(data))+'"')

Day 2 part 1: checksum is 7192

Day 2 part 2: common letters for the matching IDs are "mbruvapghxlzycbhmfqjonsie"



## [Day 3: No Matter How You Slice It](http://adventofcode.com/2018/day/3)


The Elves managed to locate the chimney-squeeze prototype fabric for
Santa's suit (thanks to <span title="WAS IT YOU">someone</span> who
helpfully wrote its box IDs on the wall of the warehouse in the middle
of the night). Unfortunately, anomalies are still affecting them -
nobody can even agree on how to *cut* the fabric.

The whole piece of fabric they're working on is a very large square - at
least `1000` inches on each side.

Each Elf has made a *claim* about which area of fabric would be ideal
for Santa's suit. All claims have an ID and consist of a single
rectangle with edges parallel to the edges of the fabric. Each claim's
rectangle is defined as follows:

  - The number of inches between the left edge of the fabric and the
    left edge of the rectangle.
  - The number of inches between the top edge of the fabric and the top
    edge of the rectangle.
  - The width of the rectangle in inches.
  - The height of the rectangle in inches.

A claim like `#123 @ 3,2: 5x4` means that claim ID `123` specifies a
rectangle `3` inches from the left edge, `2` inches from the top edge,
`5` inches wide, and `4` inches tall. Visually, it claims the square
inches of fabric represented by `#` (and ignores the square inches of
fabric represented by `.`) in the diagram below:

    ...........
    ...........
    ...#####...
    ...#####...
    ...#####...
    ...#####...
    ...........
    ...........
    ...........

The problem is that many of the claims *overlap*, causing two or more
claims to cover part of the same areas. For example, consider the
following claims:

    #1 @ 1,3: 4x4
    #2 @ 3,1: 4x4
    #3 @ 5,5: 2x2

Visually, these claim the following areas:

    ........
    ...2222.
    ...2222.
    .11XX22.
    .11XX22.
    .111133.
    .111133.
    ........

The four square inches marked with `X` are claimed by *both `1` and
`2`*. (Claim `3`, while adjacent to the others, does not overlap either
of them.)

If the Elves all proceed with their own plans, none of them will have
enough fabric. *How many square inches of fabric are within two or more
claims?*

### Part 2
Amidst the chaos, you notice that exactly one claim doesn't overlap by even a single square inch of fabric with any other claim. If you can somehow draw attention to it, maybe the Elves will be able to make Santa's suit after all!

For example, in the claims above, only claim 3 is intact after all claims are made.

What is the ID of the only claim that doesn't overlap?


In [6]:
def d3_parse_claims(data):
    '''created a list of namedtuples claim(id, x, y, w, h) from the input data'''
    claim = namedtuple('claim','id x y w h')
    claims = [ints(d) for d in data.split('\n')]
    claims = [claim(*clm) for clm in claims]
    return claims

def d3_claim_fabric(claims):
    '''create a dict from the claims that has a list of every claim for every square inch'''
    fabric = defaultdict(list)
    for claim in claims:
        for x, y in product(range(claim.x,claim.x+claim.w),
                            range(claim.y,claim.y+claim.h)):
            fabric[(x,y)].append(claim.id)
    return fabric

def day3_part1(claims):
    """find number of square inches with multiple claims"""
    claims = d3_parse_claims(claims)
    fabric = d3_claim_fabric(claims)
    return ilen(k for k,v in fabric.items() if len(v) > 1)

def day3_part2(claims):
    '''find only claim that does not share its area with other claims'''
    claims = d3_parse_claims(claims)
    fabric = d3_claim_fabric(claims)
    num_overlaps = defaultdict(int)
    for sq_claims in fabric.values():
        for sq_claim in sq_claims:
            num_overlaps[sq_claim] = max(num_overlaps[sq_claim], len(sq_claims))
    min_overlap = min(num_overlaps, key=num_overlaps.get)
    assert num_overlaps[min_overlap] == 1
    return min_overlap

In [7]:
testdata = '''\
#1 @ 1,3: 4x4
#2 @ 3,1: 4x4
#3 @ 5,5: 2x2\
'''
assert day3_part1(testdata) == 4
assert day3_part2(testdata) == 3

In [8]:
data = Input(3)
print_result(3,1,''+str(day3_part1(data)))
print_result(3,2,''+str(day3_part2(data)))

Day 3 part 1: 113576

Day 3 part 2: 825



## [Day 4: Repose Record](http://adventofcode.com/2018/day/4)

You've
<span title="Yes, &#39;sneaked&#39;. &#39;Snuck&#39; didn&#39;t appear in English until the 1800s.">sneaked</span>
into another supply closet - this time, it's across from the prototype
suit manufacturing lab. You need to sneak inside and fix the issues with
the suit, but there's a guard stationed outside the lab, so this is as
close as you can safely get.

As you search the closet for anything that might help, you discover that
you're not the first person to want to sneak in. Covering the walls,
someone has spent an hour starting every midnight for the past few
months secretly observing this guard post\! They've been writing down
the ID of *the one guard on duty that night* - the Elves seem to have
decided that one guard was enough for the overnight shift - as well as
when they fall asleep or wake up while at their post (your puzzle
input).

For example, consider the following records, which have already been
organized into chronological order:

    [1518-11-01 00:00] Guard #10 begins shift
    [1518-11-01 00:05] falls asleep
    [1518-11-01 00:25] wakes up
    [1518-11-01 00:30] falls asleep
    [1518-11-01 00:55] wakes up
    [1518-11-01 23:58] Guard #99 begins shift
    [1518-11-02 00:40] falls asleep
    [1518-11-02 00:50] wakes up
    [1518-11-03 00:05] Guard #10 begins shift
    [1518-11-03 00:24] falls asleep
    [1518-11-03 00:29] wakes up
    [1518-11-04 00:02] Guard #99 begins shift
    [1518-11-04 00:36] falls asleep
    [1518-11-04 00:46] wakes up
    [1518-11-05 00:03] Guard #99 begins shift
    [1518-11-05 00:45] falls asleep
    [1518-11-05 00:55] wakes up

Timestamps are written using `year-month-day hour:minute` format. The
guard falling asleep or waking up is always the one whose shift most
recently started. Because all asleep/awake times are during the midnight
hour (`00:00` - `00:59`), only the minute portion (`00` - `59`) is
relevant for those events.

Visually, these records show that the guards are asleep at these times:

    Date   ID   Minute
                000000000011111111112222222222333333333344444444445555555555
                012345678901234567890123456789012345678901234567890123456789
    11-01  #10  .....####################.....#########################.....
    11-02  #99  ........................................##########..........
    11-03  #10  ........................#####...............................
    11-04  #99  ....................................##########..............
    11-05  #99  .............................................##########.....

The columns are Date, which shows the month-day portion of the relevant
day; ID, which shows the guard on duty that day; and Minute, which shows
the minutes during which the guard was asleep within the midnight hour.
(The Minute column's header shows the minute's ten's digit in the first
row and the one's digit in the second row.) Awake is shown as `.`, and
asleep is shown as `#`.

Note that guards count as asleep on the minute they fall asleep, and
they count as awake on the minute they wake up. For example, because
Guard \#10 wakes up at 00:25 on 1518-11-01, minute 25 is marked as
awake.

If you can figure out the guard most likely to be asleep at a specific
time, you might be able to trick that guard into working tonight so you
can have the best chance of sneaking in. You have two strategies for
choosing the best guard/minute combination.

*Strategy 1:* Find the guard that has the most minutes asleep. What
minute does that guard spend asleep the most?

In the example above, Guard \#10 spent the most minutes asleep, a total
of 50 minutes (20+25+5), while Guard \#99 only slept for a total of 30
minutes (10+10+10). Guard \#*10* was asleep most during minute *24* (on
two days, whereas any other minute the guard was asleep was only seen on
one day).

While this example listed the entries in chronological order, your
entries are in the order you found them. You'll need to organize them
before they can be analyzed.

*What is the ID of the guard you chose multiplied by the minute you
chose?* (In the above example, the answer would be `10 * 24 = 240`.)

### Part Two

*Strategy 2:* Of all guards, which guard is most frequently asleep on
the same minute?

In the example above, Guard \#*99* spent minute *45* asleep more than
any other guard or minute - three times in total. (In all other cases,
any guard spent any minute asleep at most twice.)

*What is the ID of the guard you chose multiplied by the minute you
chose?* (In the above example, the answer would be `99 * 45 = 4455`.)


In [62]:
def d4_parse_shifts(data):
    data = sorted(data.splitlines())
    guard_buffer = []
    for entry in data:
        if 'Guard' in entry and guard_buffer:
            yield guard_buffer
            guard_buffer = [entry]
        else:
            guard_buffer.append(entry)
    yield guard_buffer

def d4_get_sleep_schedule(shifts):
    sleep_schedule = namedtuple('sleep', 'date guard minutes')
    for shift in shifts:
        line = shift.pop(0)
        guard = int(re.findall(r'Guard #(\d+)', line)[0])
        date = ''
        sleep = list()
        for down, up in zip(*([iter(shift)]*2)):
            Y, M, D, _, m_s = ints(down.replace('-','_'))
            _, _, _, _, m_w = ints(up)
            sleep.extend(range(m_s, m_w))
        yield sleep_schedule((Y,M,D), guard, sleep)

def day4_part1(data):
    """ """
    sleep_schedule = d4_get_sleep_schedule(d4_parse_shifts(data))
    guard_sleep = defaultdict(list)
    for _, guard, minutes in sleep_schedule:
        guard_sleep[guard].extend(minutes)
    sleepy_guard = max(guard_sleep, key=lambda g:len(guard_sleep[g]))
    sleepy_minute = Counter(guard_sleep[sleepy_guard]).most_common(1)[0][0]
    return sleepy_guard * sleepy_minute

def day4_part2(data):
    """ """
    sleep_schedule = d4_get_sleep_schedule(d4_parse_shifts(data))
    guard_sleep = defaultdict(list)
    for _, guard, minutes in sleep_schedule:
        guard_sleep[guard].extend(minutes)
    guard_sleepy_minute = {guard: first(Counter(sleep).most_common(1),default=(0,0))
                           for guard, sleep in guard_sleep.items()}
    sleepy_guard = max(guard_sleepy_minute, key=lambda g:guard_sleepy_minute[g][1])
    sleepy_minute = guard_sleepy_minute[sleepy_guard][0]
    return sleepy_guard * sleepy_minute

In [63]:
testdata = '''
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
'''.strip()
day4_part2(testdata)
assert day4_part1(testdata) == 240
assert day4_part2(testdata) == 4455

In [64]:
data = Input(4)
print_result(4,1,''+str(day4_part1(data)))
print_result(4,2,''+str(day4_part2(data)))

Day 4 part 1: 74743

Day 4 part 2: 132484



## [Day 5: Alchemical Reduction](http://adventofcode.com/2018/day/5)

You've managed to sneak in to the prototype suit manufacturing lab. The
Elves are making decent progress, but are still struggling with the
suit's size reduction capabilities.

While the very latest in 1518 alchemical technology might have solved
their problem eventually, you can do better. You scan the chemical
composition of the suit's material and discover that it is formed by
extremely long [polymers](https://en.wikipedia.org/wiki/Polymer) (one of
which is
<span title="I&#39;ve always wanted a polymer!">available</span> as your
puzzle input).

The polymer is formed by smaller *units* which, when triggered, react
with each other such that two adjacent units of the same type and
opposite polarity are destroyed. Units' types are represented by
letters; units' polarity is represented by capitalization. For instance,
`r` and `R` are units with the same type but opposite polarity, whereas
`r` and `s` are entirely different types and do not react.

For example:

  - In `aA`, `a` and `A` react, leaving nothing behind.
  - In `abBA`, `bB` destroys itself, leaving `aA`. As above, this then
    destroys itself, leaving nothing.
  - In `abAB`, no two adjacent units are of the same type, and so
    nothing happens.
  - In `aabAAB`, even though `aa` and `AA` are of the same type, their
    polarities match, and so nothing happens.

Now, consider a larger example, `dabAcCaCBAcCcaDA`:

    dabAcCaCBAcCcaDA  The first 'cC' is removed.
    dabAaCBAcCcaDA    This creates 'Aa', which is removed.
    dabCBAcCcaDA      Either 'cC' or 'Cc' are removed (the result is the same).
    dabCBAcaDA        No further actions can be taken.

After all possible reactions, the resulting polymer contains *10 units*.

*How many units remain after fully reacting the polymer you scanned?*
<span class="quiet">(Note: in this puzzle and others, the input is
large; if you copy/paste your input, make sure you get the whole
thing.)</span>

Your puzzle answer was `9526`.

The first half of this puzzle is complete\! It provides one gold star:
\*

### Part Two

Time to improve the polymer.

One of the unit types is causing problems; it's preventing the polymer
from collapsing as much as it should. Your goal is to figure out which
unit type is causing the most problems, remove all instances of it
(regardless of polarity), fully react the remaining polymer, and measure
its length.

For example, again using the polymer `dabAcCaCBAcCcaDA` from above:

  - Removing all `A`/`a` units produces `dbcCCBcCcD`. Fully reacting
    this polymer produces `dbCBcD`, which has length 6.
  - Removing all `B`/`b` units produces `daAcCaCAcCcaDA`. Fully reacting
    this polymer produces `daCAcaDA`, which has length 8.
  - Removing all `C`/`c` units produces `dabAaBAaDA`. Fully reacting
    this polymer produces `daDA`, which has length 4.
  - Removing all `D`/`d` units produces `abAcCaCBAcCcaA`. Fully reacting
    this polymer produces `abCBAc`, which has length 6.

In this example, removing all `C`/`c` units was best, producing the
answer *4*.

*What is the length of the shortest polymer you can produce* by removing
all units of exactly one type and fully reacting the result?

In [48]:
def d5_does_react(a, b):
    return a.lower() == b.lower() and a != b


def d5_remove_unit(polymer, unit):
    return cat(u for u in polymer if u.lower() != unit.lower())


def day5_part1(s):
    s_old = ''
    while s_old != s:
        s_old = s
        for i in range(len(s) - 1):
            a, b = s[i], s[i + 1]
            if d5_does_react(a, b):
                s = s[:i] + '##' + s[i + 2:]
        s = s.replace('##', '')
    return len(s)


def day5_part2(data):
    units = set(data.lower())
    results = []
    for unit in units:
        s = d5_remove_unit(data, unit)
        results.append(day5_part1(s))
    return min(results)

In [49]:
testdata = "dabAcCaCBAcCcaDA"
assert day5_part1(testdata) == len('dabCBAcaDA')
assert day5_part2(testdata) == 4

In [50]:
data = Input(5)
print_result(5,1,''+str(day5_part1(data)))
print_result(5,2,''+str(day5_part2(data)))

Day 5 part 1: 9526

Day 5 part 2: 6694



## [Day 6](http://adventofcode.com/2018/day/6)

In [ ]:
data = Input(3)

def day6_part1(data):
    """ """
    pass

def day6_part2(data):
    """ """
    pass

In [ ]:
testdata = '''

'''.strip()
assert day6_part1(testdata) == None
assert day6_part2(testdata) == None

In [ ]:
data = Input(6)
print_result(6,1,''+str(day6_part1(data)))
print_result(6,2,''+str(day6_part2(data)))